資料17,166筆，正確率約70%

In [1]:
import pandas as pd
import json
import sys
import os
import gensim
import numpy as np
import glob
from gensim.models.doc2vec import Doc2Vec

C:\Users\Big data\AppData\Local\Programs\Python\Python36\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
#load json檔案成dataframe,看一下內容
with open('E:/news_project/bgLabel/angle/opinion_printed_all_v2.json', 'r',encoding='UTF-8') as f:
    data = json.load(f)
    df1 = pd.DataFrame.from_dict(data, orient='columns')

df1 = df1.drop(["author"], axis=1)
df1 = df1.drop(["head"], axis=1)
df1 = df1.drop(["date_"], axis=1)
df1 = df1.drop(["type"], axis=1)
df1 = df1.drop(["source"], axis=1)

In [3]:
#丟掉不要的欄位，合併成一個datafram
df_all = pd.DataFrame(columns=['content','tag', 'title'])
for files in glob.glob('E:/news_project/bgLabel/all/*'):
    with open(files,"r", encoding="utf-8") as json_data:
        data = json.load(json_data)
        df = pd.DataFrame.from_dict(data, orient='columns')
    
        df = df.drop(["infor"], axis=1)
        df = df.drop(["date_"], axis=1)
        df = df.drop(["type"], axis=1)
        df = df.drop(["source"], axis=1)

        df_all=pd.concat([df_all,df],axis=0,ignore_index=True )
df_all=pd.concat([df_all,df1],axis=0,ignore_index=True )


In [13]:
print(df_all.columns)
df=df_all

,content,tag,title
0,民進 黨 的 主席 選戰 ， 儼 已成 為 「 保皇派 」 和 「 反 保皇派 」 之爭 。...,0,女皇的新衣
1,葉俊榮 在 宣布 教育部 將發台 大校 長 聘書給 管中 閔 之前 ， 據 說 曾 致電 立...,0,葉部長「勇敢」的代價
2,拗 了 又 拗 ， 教育部 長 葉俊榮終 於 宣布 將發台 大校 長 聘書給 管中 閔 ， ...,0,三百天才找回理性
3,蘇貞昌將 接閣 揆 已 經傳 了 一陣子 。 儘 管 其 幕僚 否認 此 說 ， 但 多位 ...,0,聽說蘇貞昌接閣揆
4,柯文 哲雙 城論壇 再 提 「 兩岸 一家 親 」 ， 又 吹 皺政壇 春水 。 對照 他 ...,0,柯文哲的如意算盤
5,民眾 對民 進黨 投下 不 信任票 ， 除了 政策 失當 ， 官員 人品 太差 也 是 重要...,0,政客 外交官 賈新文
6,看起 來 ， 民進 黨敗 選 後 真的 有 在 反省 。 繼計畫 推出 廉價 菸 、 取消 ...,0,補貼再補貼，拚經濟？
7,國人談 非洲 豬瘟而色 變 。 蔡總統 也 發表談 話 ， 強調 防疫 須 全民 總動員 。...,0,小英罵豬，意在…
8,民進 黨 檢討敗 選和同 婚議題 ， 綠委尤 美女 成眾 矢 之 的 ， 據傳 總召 柯建銘...,0,拜託民進黨放了老百姓
9,賴清德 就任 一年 多 ， 內閣 出現 了 一個 奇景 ： 撇開 政務委員 和 虛級化 的 ...,0,坑坑洞洞的行政院


In [5]:
def split_news(news):
    return " ".join(jieba.cut(news))

def process_news(df):
    df['content'] = df ['content'].apply(split_news)
    df['title'] = df['title'].str.replace('\r','')
    df['title'] = df['title'].str.replace('\n','')
    return df

In [6]:
import jieba
df = process_news(df)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\BIGDAT~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.690 seconds.
Prefix dict has been built succesfully.


In [12]:
#取出content列為一個list
b=list(df['content'])

['民進 黨 的 主席 選戰 ， 儼 已成 為 「 保皇派 」 和 「 反 保皇派 」 之爭 。 「 反派 」 的 游盈隆 人 單勢 孤 ， 在 派系 共治 的 現實 下要 突圍 ， 恐怕 是 難 上 加難 。 但 他 痛批 「 保皇 黨 」 這 一招 ， 仍 讓 民進 黨 「 女皇 的 新衣 」 被 赤裸裸 攤 在 大眾 面前 。 敗選 當夜 ， 蔡 英文 即 宣布 辭去 民進 黨 主席 。 然而 ， 「 中生代 接班 」 的 呼聲 未 歇 ， 已傳出 總統 屬意 由 立法院 長 蘇嘉全 接黨 主席 。 這項 安排 隨即 遭到 外界 批評 ， 幾經 波折 ， 又 由府 黨 高層 「 火鍋會 」 敲定 卓榮泰 出征 。 至此 ， 民進 黨 「 後 蔡 英文 時代 」 由 誰 接班 ， 應 已大勢 底定 。 但 黨 內 許多人 仍 吞 不下 這樣 的 安排 。 游盈隆 跳出 來 怒斥 卓榮泰 等 人 為 「 保皇派 」 ， 逼得 卓榮泰 不得不 跟 進 ， 宣稱 總統 沒有 「 連任 優先權 」 ， 還說 「 保皇 」 一說 是 「 沉重 的 標 籤 」 。 游盈隆 烏 鴉當 到底 ， 索性 再 以 「 獅子 和 狐狸 」 ， 暗諷 黨 內 派系 成群 結隊 ， 拉幫成 派 ， 把 綠營 高層 企圖 營造 的 大團結 面具 完全 撕下 。 卓榮泰 表面 上 代表 「 中生代 接班 」 ， 但 真正 的 因素 ， 是 卓榮泰 不具 政治 威脅 ， 各 派系 又 可 接受 ， 且 能 配合 「 保蔡 」 連任 。 大敗 流血 未止 ， 綠營 仍 心 繫 如何 牢牢 掌握 權力 ， 真是 苦心 孤詣 。 但 女皇 依舊 是 女皇 ， 只不過 多 了 卓榮泰 這件 新衣 。 這番 做作 ， 對 照敗 選之夜 蔡 英文 、 賴清德 、 陳 菊 同時 請 辭 ， 真是 一場 大戲 。 真相 是 ： 原來 主席 可以 指定 接班人 ， 閣揆 等 「 穩定 政局 」 才 辭 ， 秘書長 則文風 不動 。 所謂 敗選 負責 ， 原來 如此 。',
 '葉俊榮 在 宣布 教育部 將發台 大校 長 聘書給 管中 閔 之前 ， 據 說 曾 致電 立委 ， 稱 自己 「 做 了 一個 勇敢 的 決定 」 。 葉部長 什麼 地方 「 勇敢 」 ？ 多數民眾 會 有 此 疑問 ， 因為 「 正義遲

In [8]:
TaggededDocument=gensim.models.doc2vec.TaggedDocument

In [9]:
x_train=[]
for i, text in enumerate(b):
    word_list = text.split(' ')
    l = len(word_list)
    word_list[l-1]=word_list[l-1].strip()
    document=TaggededDocument(word_list,tags=[i])
    x_train.append(document)
#print(x_train)
c = x_train

In [10]:
model=Doc2Vec(x_train, min_count=1, window=3, vector_size=100, negative=5,workers=4)
model.train(x_train,total_examples=model.corpus_count,epochs=10)

In [11]:
model_dm=model

In [14]:
strl=df.iloc[78,0]
test_text=strl.split(' ')
#得到向量array命名為inferred_vector
inferred_vector=model_dm.infer_vector(doc_words=test_text,alpha=0.025,steps=500)

sims=model_dm.docvecs.most_similar([inferred_vector],topn=3)
sims

C:\Users\Big data\AppData\Local\Programs\Python\Python36\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[(78, 0.810858964920044),
 (3510, 0.5051697492599487),
 (3180, 0.497667133808136)]

In [15]:
#找到與特定文章strl相似的文章

for count, sim in sims:
    print(count,sim)
    sentence = c[count]
    words = ''
    
    for word in sentence[0]:
        words = words + word +' '
    print(words,sim,len(sentence[0]))

78 0.810858964920044
新任 調查 局長 呂文忠 上任 ， 一席 將 全力 調查 危害 國家 安全 假新聞 的 談話 ， 讓 人民 感到 一陣 寒風 。 將國安 如此 無限 上綱 ， 怎 不 令人 憂心 調查局 走向 效忠 政黨 ， 而 非 效忠 國家 的 回頭路 ？ 調查 局源 於 國民 政府 時期 的 「 軍統 」 與 國民黨 「 中統 」 ， 政府 遷台 後 負責 保防 偵查 ， 在 戒 嚴時期 成為 政治 打手 而 惡名 昭彰 。 姑不論 其 時代 背景 ， 這一頁 「 東廠 」 局史絕 不 光彩 ， 也 逼使 調查 局展 開漫長 的 「 國家化 」 歷程 ， 致力 肅貪 、 掃毒 、 查賄 等 ， 才 重拾 人民 信賴 。 如今 調查局 官網 明白 揭示 兩個 維護國安 工作 重點 ： 對受 中共 派遣 或 運用 ， 在 台灣 從 事 涉密 情 蒐 、 發展 組織 或 進行 破壞 的 個 人 或 組織 ， 以及 境內 外籍人士 或團體 蒐集 政經 、 國防 、 高科技 機密 資料 等 行為 ， 進行 監控 與 偵處 。 規範 如此 詳細 ， 實著眼 於 公權力 何其 強大 ， 自須 有 具體 違 法事 證作 為 調查 前提 ， 不容 單憑 「 國家 安全 」 的 不 確定 法律 概念 入人 於 罪 ； 並且 ， 所謂 危害 國安假 新聞 一旦 成為執 政者 遮掩 失政 的 卸責話術 ， 對國家 危害 實遠 甚假 新聞 。 二 ○ 一二 總統 大選前 ， 不過 是 一則 國安會疑 透過 調查局 對 蔡 英文 陣營 監控 的 傳聞 報導 ， 即 引發 其 強烈 批判 ； 如今 掌權後竟 想 假調 查局 之手 、 藉 國安之名 開啟 更廣 更深 的 干 預言論 自由 之門 ， 蔡 政府 心態 確實 可議 。 「 我們 是 國家 的 調查員 … 」 ， 展抱 山莊 激昂 的 「 調查員 進行曲 」 依舊 繞梁 ， 但展 抱 人 可 真 甘心 淪為 政治 打手 ， 讓 「 老牌 東廠 復 活 」 的 烏雲罩 頂 ？  0.810858964920044 316
3510 0.5051697492599487
去年 5 月 20 日民進 黨 全面 執政以 來 ， 兩岸關 係 江河日下 ， 重大 政策 反 反覆 覆 ， 施政 跌跌撞撞 ， 民怨 日深 

In [17]:
df['d2v_Vec']='0'
for i in range(0,17167):
    strl=df.iloc[i,0]
    test_text=strl.split(' ')
    #得到向量array命名為inferred_vector
    inferred_vector=model_dm.infer_vector(doc_words=test_text,alpha=0.025,steps=500)
    
    df.loc[i,'d2v_Vec']=list(inferred_vector)
    
print(df.head())    

                                             content tag       title  \
0  民進 黨 的 主席 選戰 ， 儼 已成 為 「 保皇派 」 和 「 反 保皇派 」 之爭 。...   0       女皇的新衣   
1  葉俊榮 在 宣布 教育部 將發台 大校 長 聘書給 管中 閔 之前 ， 據 說 曾 致電 立...   0  葉部長「勇敢」的代價   
2  拗 了 又 拗 ， 教育部 長 葉俊榮終 於 宣布 將發台 大校 長 聘書給 管中 閔 ， ...   0    三百天才找回理性   
3  蘇貞昌將 接閣 揆 已 經傳 了 一陣子 。 儘 管 其 幕僚 否認 此 說 ， 但 多位 ...   0    聽說蘇貞昌接閣揆   
4  柯文 哲雙 城論壇 再 提 「 兩岸 一家 親 」 ， 又 吹 皺政壇 春水 。 對照 他 ...   0    柯文哲的如意算盤   

                                             d2v_Vec  
0  [-2.5054448, -0.98734796, -1.4825997, -1.54023...  
1  [-1.6271399, -0.5152588, 0.76503277, -2.021135...  
2  [-1.8977276, 0.8613036, -0.82015616, -0.718078...  
3  [0.3384848, 2.0811343, -0.9328736, -1.9614244,...  
4  [2.101391, -1.7982857, -3.108805, -4.4197197, ...  


In [34]:
from sklearn.model_selection import train_test_split
# axis = 1 drop行 axis = 0 drop列
# (題目90%, 題目10%, 答案90%, 答案10%)
df_train, df_test, tag_train, tag_test = train_test_split(df, 
                                   df["tag"], test_size=0.1)

In [31]:
df_train

,content,tag,title,d2v_Vec
12953,台灣 超過 三成 大專 院校 簽下 一中 承諾書 ， 對此 台灣 依舊 眾聲 喧 譁 ， 綠...,1,各位大學校長，認識Kingman Brewster Jr.嗎？,"[0.9796089, 1.830433, 2.4289725, 1.2172141, 2...."
1399,看見 即將 結婚 的 新人 哭 得 淚流 滿臉 ， 的 確是 令人 滿 同情 的 畫面 。 ...,0,新娘不要哭,"[-0.3123678, 1.7826968, 2.6952815, -0.35341147..."
584,音樂 遇上 政治 ， 就 像 雞 蛋 碰 石頭 ， 注定 難逃 噩運 。 Ｎ Ｃ Ｃ 一紙 ...,0,音樂網好大的罪名,"[-0.8231997, -0.10672909, -0.221741, -3.621578..."
16887,今年 九月 習近 平訪華府 ， 要 美國 逼 蔡 英文 接受 九二 共識 之 說 甚囂塵 上...,1,竊據台灣 增修條文不合憲,"[-1.8706762, 0.4908798, 0.9176018, -1.8111802,..."
7828,春節 期間 ， 台灣 電影 賣座 冠軍是 豬哥亮 主演 的 「 大尾 鱸 鰻 」 ， 票房 ...,0,文創：想像無界限，但專業有門檻,"[1.7576109, -1.569177, 1.2268043, -2.5147316, ..."
4740,一個 多世紀 ， 台灣 受 外力 主宰 ， 命運 坎坷 ， 現在 一千八百 多萬選民 總算 ...,1,《鏗鏘集》用選票完成寧靜革命！,"[-1.3252753, 0.26247576, -0.4002381, -2.014028..."
675,蔡總統 過境 美國 ， 川普團隊 七日 說 ， 川普 本人 與 團隊 成員 都 不會 與 她...,0,地動山搖！誰動誰搖？,"[0.579561, -0.34544313, 0.44918233, -2.8698187..."
11266,行政院 長 賴清德 在 立院 答詢 時 表示 ， 他 是 主張 台灣 獨立 的 政治工作者 ...,1,主權獨立國家要正名,"[0.08410225, -1.5862112, 2.6253808, -1.5254401..."
7027,美國牙醫 非法 獵殺 辛巴威 獅王 塞西爾 事件 ， 引起 全球 公憤 ， 這股 怒氣化 為...,0,你獵殺，我禁運：現代企業的道德實踐,"[1.0762341, -1.0063382, 3.8994017, 2.213363, -..."
2474,台灣 有 群人 早 在 2011 年馬英 九 和 蔡 英文 競選 總統 時 ， 便 已 透過...,0,《熱門話題》台灣的另類奇蹟,"[-0.17888086, -2.6241167, 0.6851758, 1.1587824..."


In [35]:
#分類器
from sklearn.neighbors import KNeighborsClassifier
clf=KNeighborsClassifier(n_neighbors=8)
clf=clf.fit(list(df_train['d2v_Vec']),tag_train)

In [36]:
#開始預測
from sklearn.metrics import accuracy_score
predict=clf.predict(list(df_test['d2v_Vec']))
print("預測 :", predict)
print("正確標籤 :", tag_test)
print("正確率 :",accuracy_score(tag_test,predict)*100,"%")

預測 : ['0' '0' '1' ... '0' '1' '1']
正確標籤 : 3506     0
909      0
1982     0
3601     0
3912     0
12707    1
1877     0
16228    1
757      0
63       0
14771    1
3123     0
11309    1
16988    1
13244    1
12835    1
11321    1
9982     1
6125     0
5931     0
5884     0
16085    1
3368     0
355      0
13157    1
1287     0
2500     0
749      0
14705    1
14602    1
        ..
5602     1
17008    1
8392     1
131      0
6995     0
11899    1
12541    1
1310     0
15988    1
7758     0
3897     0
380      0
10295    1
8090     1
8705     1
3388     0
7170     0
5625     1
5522     1
13270    1
15191    1
8172     1
9743     1
1765     0
1760     0
8492     1
6231     0
17068    1
4971     1
8712     1
Name: tag, Length: 1717, dtype: object
正確率 : 68.20034944670937 %


In [37]:
#使用混淆矩陣，確認一下分類錯誤
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(tag_test,predict)
pd.DataFrame(cm)

,0,1
0,445,196
1,350,726


In [46]:
#隨便找一筆個資料來預測

# #綠
#strl='廿二位新任縣市首長週二宣誓就職，朝野版圖重新洗牌，國民黨掌握了十五個執政縣市，還有十九席的地方議會議長，重新回到二千年第一次政黨輪替以前的榮景，甚至更大更廣，人事任用權「隨我歡喜」，預算額度「大碗滿墘」，以國民黨人傳統的習性，這麼一桌桌滿漢全席吃下來，未來會不會體態臃腫、弊端叢生？恐怕會忙壞檢調囉！這一次地方大選，蔡英文徹底失敗了，敗的和馬英九執政時一樣的一塌糊塗，縱使原因有千百種，仍然看不到這個政府自我檢討，「討厭民進黨」成了台灣最大黨，必然還會延續一段時間。在此同時，選民並未同時「喜歡國民黨」，國民黨勝之不武，撿到天上掉下來的禮物，應該選輸的候選人卻已贏得大位。這也是標榜「超越藍綠」的柯文哲仍然屹立不搖的原因。除了「討厭民進黨」之外，這次選舉結果還有一個特色是，「國民黨的議會回來了！」幾十年來銀彈發揮威力的地方議會，竟然因為民眾討厭民進黨而起死回生。回鍋的回鍋，接棒的接棒，地方議會幾乎又回到李登輝被痛批「黑金政治」的時代。當議員要幹嘛？為民服務嘛！為民服務要銀兩，錢從哪裡來？還不是從預算掏出來，靠派系、靠議員抬轎的縣市長「準備」好了嗎？整碗、整桌都準備好了嗎？千萬別小看縣市長，他們有權准駁許多開發計畫，六都市長掌控的年度預算金額可都是天文數字，最多的台北市有一千七百億元，新北市有一千六百億元，台中市和高雄市也都有一千三百億元。其他縣長平均也有幾百億元的預算。為了配合「具有國民黨特色的議會」的民意，縣市政府要不要編預算配合議員？未來會不會蓋出更多的「蚊子館」及「馬奮館」？議會看緊人民荷包還是把荷包掏空？不免令人充滿想像。縣市長掌握的人事權也受矚目，宣稱「聖人難找」的高雄市長韓國瑜，一度原想任命的原民會主委吸金，海洋局長涉賄當選無效，他任命的教育局長曾被彈劾處分，盧秀燕的財政局長有家暴紀錄。這都只是上任前的檢驗，可見當縣市長稍有偏差，人事權的行使就會出現問題。國民黨激增地方版圖，中了政治樂透，儼然像個身懷鉅款、腰纏萬貫的闊佬，大家不妨懷著酸葡萄心理，等著看他們怎麼享用「大碗擱滿墘」的大餐'

#藍
#strl="國民黨發言人洪孟楷今早宣布投入2020立委選舉，參選第10屆新北市第一選區（淡水、林口、泰山、八里、三芝、石門）立委，將有機會對戰有「淡水蔡依林」之稱的現任綠委呂孫綾。洪孟楷表示，看到中央執政的弊病不斷，現在的執政黨無心想人民，只在意如何打擊政敵，對付國民黨；《勞動基準法》亂修、年金改革違反信賴原則、前瞻計畫是撒錢自肥、兩岸關係冷凍，他所熱愛的台灣與中華民國，還有多少個4年可以浪費呢？他將用要用年輕的熱忱，打一場正向陽光選戰，要用過去10年累積的專業，監督揭弊、爭取預算，不做負面攻擊、跳脫政黨鬥爭。兼任文傳會義務職副主委的洪孟楷今正式表態參選2020立委，他說，過去馬政府8年執政，當時他與國民黨並未有過多接觸，然在2016年國民黨在野後，當眾人選擇遠離，他毅然決然擔任黨職，並且為黨辯護，捍衛，可以說國民黨越弱時、更加不離不棄，因為相信民主政治需要良好的監督制衡，才能夠給民眾更好的生活。洪孟楷說，在政論節目中他只論理、不硬拗，因為相信「真理越辯越明」，但過去國民黨有不對的地方也絕不護短；在地方服務上，更是不分黨派、不分顏色，只要是對地方好的事物，孟楷勇於付出，樂於奉獻，這是他永遠的承諾。洪孟楷說，在這次九合一選舉中看到了「討厭民進黨」的勝利，但不見得是每個人都對於國民黨放心，因此，如果接下來的選舉，國民黨還是沒有任何改變，用一樣的面孔來面對選民，這樣怎麼能夠回應民眾對於政黨的期待呢？洪孟楷說，今年36歲的他，有完整的中央、地方政府歷練，曾是前台北縣副縣長蔡家福秘書，嘉義市政府文化局局長，也是行政院前院長江宜樺辦公室專門委員，他有信心能夠為公眾爭取權利，創造地方人人發大財"
strl='蔡總統連日展開「傾聽之旅」，為民進黨敗選聽取地方意見。從形式上看，這完全是一個定位錯亂的安排。原因是，蔡英文已失去黨主席職務，她以總統身分出訪，卻只造訪綠營失去政權的縣市，且只會見綠營首長，聽一方之詞。可見，她迄今對自己的國家元首角色仍毫無「超越藍綠」的自覺。一路上，蔡英文仍維持她一貫的「英式溝通法」。她要求黨員及支持者，給指教可以「更直接、更大聲」；但對於基層的質疑，她卻一再重複令人無感的官式辯解。更可笑的是，她還為「認真做事的人沒得到連任的機會」感到不平，公開質疑選民的抉擇。試問，這是哪門子反省？又是哪門子傾聽？兩年多前蔡英文就說過，民眾的聲音如果政府沒聽見，「可以拍桌」。事實上，人民喊得再大聲，有用嗎？當群眾在街頭怒吼，政府卻用層層拒馬把自己和人民隔開；當網路上民怨四處竄燒，政府卻視為「假消息」，要採取國安手段對付。兩年來，媒體上檢討及批評施政不當的文章連篇累牘，蔡總統和她的團隊若曾聽進一半，今天民進黨還會敗成這樣嗎？於是，這樣的「傾聽之旅」會有什麼效果呢？在台中，林佳龍稱自己是敗軍之將「只能聆聽」；在雲林，李進勇懺稱自己是「頭號戰犯」；在彰化，魏明谷則抱歉再抱歉。既然地方敗將都公開認錯了，蔡英文這趟傾聽之旅不論聽到什麼，她應該都頗感「療癒」吧！'

In [47]:
#隨便找一筆個資料來預測
strl=" ".join(jieba.cut(strl))
test_text=strl.split(' ')
inferred_vector=model_dm.infer_vector(doc_words=test_text,alpha=0.025,steps=500)
ddf = pd.DataFrame(columns=['tag', 'd2v_Vec'])
ddf.loc[0,'d2v_Vec']=list(inferred_vector)

In [48]:
predict=clf.predict(list(ddf['d2v_Vec']))
print("預測 :", predict)

預測 : ['0']
